Solar Radiation Measurement Data EDA

In [13]:
# Load the data 
import pandas as pd

togo_data = pd.read_csv('../../data/data/togo-dapaong_qc.csv')
benin_data = pd.read_csv('../../data/data/benin-malanville.csv')
sierra_leone_data = pd.read_csv('../../data/data/sierraloeone-bumbuna.csv')